In [1]:
import re
import pandas as pd

import stanza
# Stanza 초기화
stanza.download('id')  # 영어 모델 다운로드
nlp = stanza.Pipeline('id', processors='tokenize')
import os
file_list=os.listdir()

import os
import pandas as pd

from tqdm import tqdm

2023-10-04 15:01:57 INFO: Downloading default packages for language: id (Indonesian) ...
2023-10-04 15:01:58 INFO: File exists: C:\Users\USER\stanza_resources\id\default.zip
2023-10-04 15:02:00 INFO: Finished downloading models and saved to C:\Users\USER\stanza_resources.
2023-10-04 15:02:00 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-10-04 15:02:00 WARNING: Language id package default expects mwt, which has been added
2023-10-04 15:02:00 INFO: Loading these models for language: id (Indonesian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |

2023-10-04 15:02:00 INFO: Using device: cpu
2023-10-04 15:02:00 INFO: Loading: tokenize
2023-10-04 15:02:00 INFO: Loading: mwt
2023-10-04 15:02:00 INFO: Done loading processors!


In [ ]:
df = pd.read_csv('C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\위키 데이터\\20230920filterwiki_문장분리전.csv')
df = df.dropna()
df

In [ ]:
def tokenizing(df):
    max_length=5
    new_rows = []
    index=0
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
        index+=1
        article_text = row['cleansing_text']
        doc = nlp(article_text)
        sen_id = 1  # 문장 ID를 1로 초기화
        for sentence in doc.sentences:
            new_row = row.copy()
            new_row['Sentence'] = sentence.text
            new_row['Token']= [token.text for token in sentence.tokens]
            new_row['Word_Count'] = len(new_row['Token'])
            new_row['Doc_ID']='20230816_wikidata_'+new_row['title']+'_000001'
            new_row['Sen_ID'] = new_row['Doc_ID']+'_sen'+str('{:06d}'.format(sen_id))
            new_row['Pub_Date']=new_row['date'][:10]
            new_row['Title']=new_row['title']
            new_row['Pub_Subj']=new_row['title']
            new_rows.append(new_row)
            sen_id += 1  # 문장 ID 증가

    new_df = pd.DataFrame(new_rows)
    new_df['Tokenized_Sentence']= new_df['Token'].apply(' '.join)
    new_df['Col_Date']='2023-08-10'
    new_df['Pub_Type']='Wikipedia'
    new_df['Text']=new_df['cleansing_text']
    new_df['Filename']='C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NIA AI 데이터셋 구축 - General/06-수집데이터/위키 데이터/rawdata/idwiki-20230720-pages-articles-multistream.xml'

    new_df.drop_duplicates(subset='Sentence', inplace=True)
    new_df.reset_index(drop=True, inplace=True)
    new_df = new_df[['Doc_ID', 'Filename', 'Title', 'Pub_Type', 'Pub_Subj', 'Pub_Date', 'Col_Date', 'Sen_ID', 'Word_Count', 'Text', 'Sentence','Tokenized_Sentence' , 'Token']]
    new_df = new_df[new_df['Token'].apply(lambda x: len(x) > max_length)]
    return new_df

In [ ]:
df_test= tokenizing(df)
df_test.head()